In [8]:
import pandas as pd
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

In [9]:
fields = ["Date_HPSC", "TotalLabs", "Test24", "PRate"]
cov_nums = pd.read_csv("LaboratoryLocalTimeSeriesHistoricView.csv", usecols=fields)
cov_nums.head()

,Date_HPSC,TotalLabs,PRate,Test24
0,2020/03/18 11:00:00+00,6457,6.1,0
1,2020/03/19 11:00:00+00,7403,6.3,946
2,2020/03/20 11:00:00+00,8872,6.4,1469
3,2020/03/21 11:00:00+00,10436,6.0,1564
4,2020/03/22 11:00:00+00,11414,6.5,978


In [10]:
def positive(row):
    pos = (row['Test24'] / 100) * row['PRate']
    return round(pos)

cov_nums['daily_pos'] = cov_nums.apply(positive, axis=1)
cov_nums = cov_nums.iloc[1:]
cov_nums.tail()

,Date_HPSC,TotalLabs,PRate,Test24,daily_pos
389,2021/04/11 11:00:00+00,4184407,5.9,14355,847
390,2021/04/12 11:00:00+00,4196325,5.9,11918,703
391,2021/04/13 11:00:00+00,4209975,5.9,13650,805
392,2021/04/14 11:00:00+00,4226847,5.9,16872,995
393,2021/04/15 11:00:00+00,4241709,5.9,14862,877


In [11]:
test_info = ["Date_HPSC", "Test24"]
test_df = cov_nums[test_info]
test_df = pd.DataFrame(test_df)
test_df['Date_HPSC'] = pd.to_datetime(test_df['Date_HPSC']).dt.date
test_df.head()

,Date_HPSC,Test24
1,2020-03-19,946
2,2020-03-20,1469
3,2020-03-21,1564
4,2020-03-22,978
5,2020-03-23,2581


In [12]:
pos = ['Date_HPSC','Test24','daily_pos','PRate']
pos_df = cov_nums[pos]
pos_df = pd.DataFrame(pos_df)
pos_df['Date_HPSC'] = pd.to_datetime(pos_df['Date_HPSC']).dt.date

In [13]:
def seq(row):
    length = (int(row['Test24']) - int(row['daily_pos']))
    daily_pos = int(row['daily_pos'])
    init_seq = [0] * length
    init_pos = [1] * daily_pos
    seq_list = init_seq + init_pos
    for i in range(len(seq_list)-1, 0, -1):
        j = random.randint(0, i+1)
        seq_list[i], seq_list[j] = seq_list[j], seq_list[i]
    return list(seq_list)

pos_df['seq_list'] = pos_df.apply(seq, axis=1)
pos_df = pd.DataFrame(pos_df)


In [14]:
def hasPositive(block):
    for node in block:
        if node == 1:
            return True
    return False

def splitBlock(block):
    halfBlock = len(block) // 2
    return block[:halfBlock], block[halfBlock:]

def binSplit(data, count):
    count += 1
    #print("Data:", data)
    if len(data) > 1 and hasPositive(data):     # Branch is complete
        left, right = splitBlock(data)      # Split the block
       # print("Left:", left, "\tRight:", right)
        if len(left) <= 1 or not hasPositive(left):     # Left branch is complete
        #    print("Left branch is complete")
            return binSplit(right, count)
        result, count = binSplit(left, count)
      #  print("Result:", result)
        if result is True:
       #     print("Result was True")
            return binSplit(right, count)
      #  print("Adding result to right and splitting")
        return binSplit((result + right), count)
    else:
     #   print("Returning True")
        return True, count

def runtime(row):
    answer = binSplit(row, 0)[1]
    return answer

pos_df['pool_test'] = pos_df['seq_list'].apply(runtime)
pos_df['pool_test'].head()
pos_df = pos_df.drop(['seq_list'], axis=1)

df = pos_df
df.head()

,Date_HPSC,Test24,daily_pos,PRate,pool_test
1,2020-03-19,946,60,6.3,405
2,2020-03-20,1469,94,6.4,562
3,2020-03-21,1564,94,6.0,582
4,2020-03-22,978,64,6.5,406
5,2020-03-23,2581,157,6.1,1013


In [15]:
pred_pool = ['Date_HPSC', 'pool_test']
pred_pool = df[pred_pool]
pred_pool.rename(columns={'Date_HPSC':'ds',
                          'pool_test':'y'},
                 inplace=True)

test_df.head()


C:\Users\evand\Anaconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,Date_HPSC,Test24
1,2020-03-19,946
2,2020-03-20,1469
3,2020-03-21,1564
4,2020-03-22,978
5,2020-03-23,2581


In [ ]:
m = Prophet.fit(pred_pool)
future = m.make_future_dataframe(peroids=30)
future.head()



In [ ]:
forecast = m.predict(future)
forecast.rename(columns={'ds':'Date_HPSC',
                         'trend': 'pred_pool'},
                inplace=True)
forecast = forecast.set_index('Date_HPSC')
forecast.head()
